# 138 Data Processing Script 🐟🐠🐡

This script takes raw source 138 data (Data Plus) and converts it into a format suitable for database import.

## Steps

### 1. Import Modules and Set Working Directories
- Define the path to your **raw source data**.
- Define the path for **outputs from data renaming and reordering**.

### 2. Rename and Reorder CSV Files
- Process the `site`, `haul`, `fish`, and `count` CSV files to match the **database import schema**.
- **Check for duplicate columns** in each dataframe.
- **Print column headers and number of rows** to cross-check with the original data.

### 2.5 Expand Count Data
- Transform `count` data so that **each counted fish receives its own unique `enc_key`** and row of data.

### 3. Prepare Staging Tables
- Organize cleaned and restructured data into **import-ready tables** for `site`, `haul`, `fish`, `count`, and `encounters`.

### 4. Save Output
- Save all cleaned and staged tables to the **defined output paths**.
- This final output path is seperate from the inputs and outputs from the data cleaning and reordering


# 1. Import Modules and Set Working Directories

In [ ]:
import sys
import pandas as pd
import numpy as np
from pathlib import Path


# Add project root to Python path so we can import schema_config
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

In [ ]:
from schema_config import (
    STAGING_SITE_COLUMNS,
    STAGING_HAUL_COLUMNS,
    STAGING_ENCOUNTER_COLUMNS,
    RAW_TO_DB_MAPPING
)

#sys.path.insert(0, str(project_root))

In [ ]:
# Set relative filepaths
notebook_dir = Path.cwd()
project_root = notebook_dir.parent

input_data_dir = project_root / 'data/processed_data'
output_data_dir = project_root / 'data/clean_data'

#print(input_data_dir)
print(input_data_dir / 'count_readable.csv')

In [ ]:
# Read CSVs into dictionary
dfs = {file.stem: pd.read_csv(file) for file in input_data_dir.glob('*.csv')}
df_count = dfs['count_readable']
df_fish  = dfs['fish_readable']
df_haul  = dfs['haul_readable']
df_site  = dfs['site_readable']

print("Data loaded successfully.")

# 2. Rename and Reorder CSV Files to match db schema
- Process the `site`, `haul`, `fish`, and `count` CSV files to match the **database import schema**.
- **Check for duplicate columns** in each dataframe.
- **Print column headers and number of rows** to cross-check with the original data.
  
### lines with # exist in the raw data but no match in the db

## Site

In [ ]:
# Rename columns
df_site_cleaned = df_site.rename(columns=RAW_TO_DB_MAPPING['site'])

# Check for duplicate columns 
duplicate_cols = df_site_cleaned.columns[df_site_cleaned.columns.duplicated()].tolist()
if duplicate_cols:
    print("⚠️ Duplicate columns found in df_site_cleaned:", duplicate_cols)
else:
    print("No duplicate columns found in df_site_cleaned.")

# Validate columns - check for missing columns, add and fill with NA
missing_cols = set(STAGING_SITE_COLUMNS) - set(df_site_cleaned.columns)
if missing_cols:
    print("\nColumns missing from site dataframe: ")
    for col in missing_cols:
        print("- ", col)

# Add missing data
df_site_cleaned['biologist'] = 'Amadon, Anna'
df_site_cleaned['org_code'] = 'UDWR-M'
df_site_cleaned['study_code'] = 138
df_site_cleaned['gear_type_code'] = 'SE'
#df_site_cleaned['record_status'] = 'Valid'

df_site_cleaned['habitat_key'] = df_site_cleaned['sample_key'].astype(str) + "_hab"     # For seining habitat_key == sample_key

conditions = [
    (df_site_cleaned['hydro_area_code'] == 'CO') & (df_site_cleaned['estimated_river_mile'] <= 115),
    (df_site_cleaned['hydro_area_code'] == 'GR') & (df_site_cleaned['estimated_river_mile'] > 128),
    (df_site_cleaned['hydro_area_code'] == 'GR') & (df_site_cleaned['estimated_river_mile'] <= 128),
]
choices = ['Lower Colorado River', 'Middle Green River', 'Lower Green River']

df_site_cleaned['site_or_reach_name'] = np.select(conditions, choices, default='Unknown')

# Reindex: adds missing columns (NaN), removes extras, reorders
df_site_cleaned = df_site_cleaned.reindex(columns=STAGING_SITE_COLUMNS)

# Save the cleaned CSV
df_site_cleaned.to_csv(output_data_dir / 'site_cleaned.csv', index=False)   # <-- actually saves the CSV

print("\ndf_site cleaned, columns renamed, and saved")
print("Columns in cleaned df_site:", df_site_cleaned.columns.tolist())
print("Number of rows in table:", len(df_site_cleaned))
# print(df_site_cleaned.head())

## Haul

In [ ]:
# Apply renaming 
df_haul_cleaned = df_haul.rename(columns=RAW_TO_DB_MAPPING['haul'])

# Check for duplicate columns 
duplicate_cols = df_haul_cleaned.columns[df_haul_cleaned.columns.duplicated()].tolist()

if duplicate_cols:
    print("⚠️ Duplicate columns found in df_haul_cleaned:", duplicate_cols)
else:
    print("No duplicate columns found in df_haul_cleaned.")

# Validate columns
missing_cols = set(STAGING_HAUL_COLUMNS) - set(df_haul_cleaned.columns)
if missing_cols:
    print("\nColumns missing from haul dataframe: ")
    for col in missing_cols:
        print("- ", col)

# Add missing data
# df_haul_cleaned['record_status'] = 'Valid'
df_haul_cleaned['ismp_seine_method'] = df_haul_cleaned['ismp_seine_method'].replace({'P': 'Parallel', 'A': 'Across'})

# Reindex: adds missing columns (NaN), removes extras, reorders
df_haul_cleaned = df_haul_cleaned.reindex(columns=STAGING_HAUL_COLUMNS)

# Save the cleaned CSV
df_haul_cleaned.to_csv(output_data_dir / 'haul_cleaned.csv', index=False)

print("\ndf_haul cleaned, columns renamed, and saved")
print("Columns in cleaned df_haul:", df_haul_cleaned.columns.tolist())
print("Number of rows in table:", len(df_haul_cleaned))

## Fish

In [ ]:
# Rename columns
df_fish_cleaned = df_fish.rename(columns=RAW_TO_DB_MAPPING['fish'])

# Check for duplicate columns 
duplicate_cols = df_fish_cleaned.columns[df_fish_cleaned.columns.duplicated()].tolist()

if duplicate_cols:
    print("⚠️ Duplicate columns found in df_fish_cleaned:", duplicate_cols)
else:
    print("No duplicate columns found in df_fish_cleaned.")

# Validate that all expected columns exist
missing_cols = set(STAGING_ENCOUNTER_COLUMNS) - set(df_fish_cleaned.columns)
if missing_cols:
    print("\nColumns missing from fish dataframe: ")
    for col in missing_cols:
        print("- ", col)

# Add required data
df_fish_cleaned['fish_count'] = 1

# Reindex: adds missing columns (NaN), removes extras, reorders
df_fish_cleaned = df_fish_cleaned.reindex(columns=STAGING_ENCOUNTER_COLUMNS)

# Save the cleaned CSV
df_fish_cleaned.to_csv(output_data_dir / 'fish_cleaned.csv', index=False)

print("\ndf_fish cleaned, columns renamed, and saved")
print("Columns in cleaned df_fish:", df_fish_cleaned.columns.tolist())
print("Number of rows in staging table:", len(df_fish_cleaned))

## Count

In [ ]:
#  Rename columns ---
df_count_cleaned = df_count.rename(columns=RAW_TO_DB_MAPPING['count'])

# Remove duplicate columns just in case ---
duplicate_cols = df_count_cleaned.columns[df_count_cleaned.columns.duplicated()].tolist()

if duplicate_cols:
    print("⚠️ Duplicate columns found in df_count_cleaned:", duplicate_cols)
else:
    print("No duplicate columns found in df_count_cleaned.")

# Validate expected columns
missing_cols = set(STAGING_ENCOUNTER_COLUMNS) - set(df_count_cleaned.columns)
if missing_cols:
    print("\nColumns missing from count dataframe: ")
    for col in missing_cols:
        print("- ", col)


# Reindex: adds missing columns (NaN), removes extras, reorders
df_count_cleaned = df_count_cleaned.reindex(columns=STAGING_ENCOUNTER_COLUMNS)

# Save the cleaned CSV
df_count_cleaned.to_csv(output_data_dir / 'count_cleaned.csv', index=False)
print("\ndf_count cleaned, columns renamed, and saved")
print("Columns in cleaned df_count:", df_count_cleaned.columns.tolist())
print("Number of rows in table:", len(df_count_cleaned))

In [ ]:
# create encounter dataframe

df_encounter_cleaned = pd.concat([df_fish_cleaned, df_count_cleaned], ignore_index=True)
df_encounter_cleaned = df_encounter_cleaned.sort_values(['haul_key', 'encounter_key']).reset_index(drop=True)

df_encounter_cleaned['record_status'] = 'Valid'

df_encounter_cleaned.to_csv(output_data_dir / 'encounter_cleaned.csv', index=False)

print(df_encounter_cleaned.head())

# 🎉 Woohoo! Data Processing Complete! 🎉

Congratulations! 🥳  